<a href="https://colab.research.google.com/github/Venchislav/Data-Science/blob/main/KNN_k_nearest_neighbors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np

In [71]:
df = pd.read_csv('/content/sample_data/telecom_churn.csv')
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [72]:
df.drop(["State", "Voice mail plan"], axis=1, inplace=True)

In [73]:
df["International plan"] = df["International plan"].map({"Yes": 1, "No": 0})
df.info()
y = df["Churn"].astype("int")
X = df.drop("Churn", axis=1)
X.shape, y.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Account length          3333 non-null   int64  
 1   Area code               3333 non-null   int64  
 2   International plan      3333 non-null   int64  
 3   Number vmail messages   3333 non-null   int64  
 4   Total day minutes       3333 non-null   float64
 5   Total day calls         3333 non-null   int64  
 6   Total day charge        3333 non-null   float64
 7   Total eve minutes       3333 non-null   float64
 8   Total eve calls         3333 non-null   int64  
 9   Total eve charge        3333 non-null   float64
 10  Total night minutes     3333 non-null   float64
 11  Total night calls       3333 non-null   int64  
 12  Total night charge      3333 non-null   float64
 13  Total intl minutes      3333 non-null   float64
 14  Total intl calls        3333 non-null   

((3333, 17), (3333,))

In [74]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=17
)
X_train.shape, X_test.shape
first_tree = DecisionTreeClassifier(random_state=17)
np.mean(cross_val_score(first_tree, X_train, y_train, cv=5))

0.9138423504976518

In [75]:
KNeighborsClassifier?

In [76]:
first_knn = KNeighborsClassifier()
np.mean(cross_val_score(first_knn, X_train, y_train, cv=5))

0.8671274043984523

In [77]:
# but this doesn't mean that knn is worse than tree here.
# I mean here it's worse...
# But not because of knn

In [78]:
tree_params = {"max_depth": np.arange(1, 11), "max_features": [0.5, 0.7, 1]}
grid = GridSearchCV(first_tree, tree_params, cv=5, n_jobs=-1)

In [79]:
%%time
grid.fit(X_train, y_train);

CPU times: user 212 ms, sys: 39.2 ms, total: 251 ms
Wall time: 2.93 s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=17), n_jobs=-1,
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'max_features': [0.5, 0.7, 1]})

In [80]:
grid.best_score_  # best cross_val score

0.9391366681677404

In [81]:
grid.best_params_

{'max_depth': 6, 'max_features': 0.7}

In [82]:
# and now let's apply grid_search_cv for KNN

In [83]:
first_knn = KNeighborsClassifier()

In [84]:
knn_params = {"n_neighbors": range(5, 30, 5)}
grid_knn = GridSearchCV(first_knn, knn_params, cv=5, n_jobs=-1)

In [85]:
%%time
grid_knn.fit(X_train, y_train);

CPU times: user 62 ms, sys: 3.19 ms, total: 65.2 ms
Wall time: 913 ms


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': range(5, 30, 5)})

In [86]:
grid_knn.best_score_  # best cross_val score

0.8701289391697531

In [87]:
grid_knn.best_params_

{'n_neighbors': 10}

In [88]:
# Yeah... KNN is worse here...
# But we scientifically proved this!

In [89]:
# as tree is better, let's train our model no it

In [90]:
clf = grid.best_estimator_

In [91]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=6, max_features=0.7, random_state=17)

In [92]:
y_pred = clf.predict(X_test)

In [93]:
from sklearn.metrics import accuracy_score

In [94]:
# base line here:
1 - np.mean(y)

0.8550855085508551

In [95]:
accuracy_score(y_test, y_pred)

0.936